<a href="https://colab.research.google.com/github/dafrie/fin-disclosures-nlp/blob/master/notebooks/ColabLabelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labelling using Colab [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dafrie/fin-disclosures-nlp/blob/master/notebooks/ColabLabelling.ipynb)

- Uses custom widgets for labelling
- Loads and stores the data in a google storage bucket

